In [ ]:
CROSS-DIMENSION ANALYSIS

3D

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression

# Leggi tutti i dataset GPT
datasets_gpt = {
    'Grammar': pd.read_csv("gr_gpt_ALL.csv"),
    'Syntax': pd.read_csv("synt_gpt_ALL.csv"),
    'Readability': pd.read_csv("read_gpt_ALL.csv"),
    'InfoDens': pd.read_csv("infdens_gpt_ALL.csv")
}

def prepare_data_for_analysis(datasets):
    all_data = []

    for dimension, df in datasets.items():
        df = df.copy()

        if dimension == 'InfoDens':
            if 'setting' in df.columns:
                df['percentage'] = df['setting'].map({'2s_3e': 50, '4s_3e': 100})
        else:
            if "sentence_percentage" in df.columns:
                df.rename(columns={"sentence_percentage": "percentage"}, inplace=True)

        df['dimension'] = dimension
        all_data.append(df)

    return pd.concat(all_data, ignore_index=True)

df_gpt = prepare_data_for_analysis(datasets_gpt)

df_filtered = df_gpt[['acc_median_final', 'comp_final_median', 'Conciseness']].copy()
df_filtered = df_filtered.dropna()
df_filtered = df_filtered[df_filtered['Conciseness'] <= 4]

# ======================
# MULTIPLE VIEWPOINTS
# =====================

def create_multiple_viewpoints():
    """Mostra il grafico 3D da 4 angolazioni diverse"""

    fig = plt.figure(figsize=(16, 14))

    viewpoints = [
        (20, 45, 'View 1'),
        (20, 135, 'View 2'),
        (30, 225, 'View 3'),
        (15, 315, 'View 4')
    ]

    acc = df_filtered['acc_median_final'].values
    comp = df_filtered['comp_final_median'].values
    conc = df_filtered['Conciseness'].values

    # Regression
    X = np.column_stack([acc, comp])
    y = conc
    model = LinearRegression()
    model.fit(X, y)

    # Mesh
    acc_range = np.linspace(acc.min(), acc.max(), 15)
    comp_range = np.linspace(comp.min(), comp.max(), 15)
    acc_mesh, comp_mesh = np.meshgrid(acc_range, comp_range)
    X_mesh = np.column_stack([acc_mesh.ravel(), comp_mesh.ravel()])
    conc_mesh = model.predict(X_mesh).reshape(acc_mesh.shape)

    for idx, (elev, azim, title) in enumerate(viewpoints, 1):
        ax = fig.add_subplot(2, 2, idx, projection='3d')

        # Plane
        ax.plot_surface(acc_mesh, comp_mesh, conc_mesh,
                       alpha=0.3, cmap='coolwarm', edgecolor='none')

        # Scatter
        ax.scatter(acc, comp, conc, c=conc, cmap='viridis',
                  s=20, alpha=0.6, edgecolors='black', linewidth=0.3)

        # Labels
        ax.set_xlabel('Accuracy', fontsize=20, fontweight='bold')
        ax.set_ylabel('Completeness', fontsize=20, fontweight='bold')
        ax.set_zlabel('Conciseness', fontsize=20, fontweight='bold')
        ax.set_title(title, fontsize=20, fontweight='bold')

        # Limits
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_zlim(0, 4)

        # Viewpoint
        ax.view_init(elev=elev, azim=azim)

        ax.grid(alpha=0.3)
        ax.tick_params(labelsize=9)

    fig.suptitle('GPT: 3D Correlation', fontsize=16, fontweight='bold', y=0.995)

    plt.tight_layout(rect=[0, 0, 1, 0.99])
    return fig

fig = create_multiple_viewpoints()
plt.show(fig)
plt.close()

GPT VS GEMINI: Correlation of All metrics

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
from itertools import combinations

# === OUTPUT FOLDER===
os.makedirs("plots_correlations", exist_ok=True)

# === FILE GPT  ===
file_openai = {
    "Grammar": "gr_gpt_ALL.csv",
    "Syntax": "synt_gpt_ALL.csv",
    "Readability": "read_gpt_ALL.csv",
    "InfoDens": "infdens_gpt_ALL.csv",
}

# === FILE GEMINI ===
file_gemini = {
    "Grammar": "gr_gemini_ALL.csv",
    "Syntax": "synt_gemini_ALL.csv",
    "Readability": "read_gemini_ALL.csv",
    "InfoDens": "infdens_gemini_ALL.csv",
}

# === EVALUATION METRICS ===
metrics = [
    "acc_median_final",
    "comp_final_median",
    "key_concept_coverage",
    "Conciseness",
    "Num_addition",
    "Num_direct_modifications"
]

metric_labels = {
    "acc_median_final": "Accuracy",
    "comp_final_median": "Completeness",
    "key_concept_coverage": "KCC",
    "Conciseness": "Conciseness",
    "Num_addition": "Additions",
    "Num_direct_modifications": "NumReplicas"
}

def prepare_df(df):
    if "sentence_percentage" in df.columns:
        df.rename(columns={"sentence_percentage": "percentage"}, inplace=True)
    if "setting" in df.columns:
        mapping = {"2s_3e": 50, "4s_3e": 100}
        df["percentage"] = df["setting"].map(mapping)
    return df

def load_all(files_dict):
    dfs = []
    for dimension, path in files_dict.items():
        df = pd.read_csv(path, encoding="utf-8")
        df = prepare_df(df)
        df["dimension"] = dimension
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

print("Loading data...")
df_gpt = load_all(file_openai)
df_gemini = load_all(file_gemini)

for m in metrics:
    df_gpt[m] = pd.to_numeric(df_gpt[m], errors="coerce")
    df_gemini[m] = pd.to_numeric(df_gemini[m], errors="coerce")

df_gpt_clean = df_gpt[metrics + ["dimension"]].dropna()
df_gemini_clean = df_gemini[metrics + ["dimension"]].dropna()

print(f"GPT samples: {len(df_gpt_clean)}")
print(f"Gemini samples: {len(df_gemini_clean)}")

dimension_colors = {
    "Grammar": "#e74c3c",
    "Syntax": "#3498db",
    "Readability": "#2ecc71",
    "InfoDens": "#f39c12"
}


# ================================
# ALL CORRELATIONS GPT VS GEMINI
# =================================


metric_pairs = list(combinations(metrics, 2))
n_pairs = len(metric_pairs)

n_cols = 3
n_rows = int(np.ceil(n_pairs / n_cols))


fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4.5))
fig.suptitle('GPT vs Gemini: Correlations Between All Metrics\n' +
             'Blue = GPT | Red = Gemini | Colors = Dimensions',
             fontsize=18, fontweight='bold', y=0.998)

axes = axes.flatten()

for idx, (metric_x, metric_y) in enumerate(metric_pairs):
    ax = axes[idx]
    label_x = metric_labels[metric_x]
    label_y = metric_labels[metric_y]

    # Plot GPT (blue)
    for dim, dim_color in dimension_colors.items():
        df_dim = df_gpt_clean[df_gpt_clean['dimension'] == dim]
        x_data = df_dim[metric_x].values
        y_data = df_dim[metric_y].values
        ax.scatter(x_data, y_data, alpha=0.3, s=25,
                  color=dim_color, marker='o', edgecolors='blue', linewidth=0.5)

    # Regression GPT
    x_gpt = df_gpt_clean[metric_x].dropna().values
    y_gpt = df_gpt_clean[metric_y].dropna().values
    min_len = min(len(x_gpt), len(y_gpt))
    if min_len > 1:
        x_gpt = x_gpt[:min_len]
        y_gpt = y_gpt[:min_len]
        slope_gpt, intercept_gpt, r_gpt, _, _ = stats.linregress(x_gpt, y_gpt)
        line_x = np.array([x_gpt.min(), x_gpt.max()])
        line_y = slope_gpt * line_x + intercept_gpt
        ax.plot(line_x, line_y, 'b-', linewidth=2.5, alpha=0.8, label=f'GPT (r={r_gpt:.2f})')

    # Plot Gemini
    for dim, dim_color in dimension_colors.items():
        df_dim = df_gemini_clean[df_gemini_clean['dimension'] == dim]
        x_data = df_dim[metric_x].values
        y_data = df_dim[metric_y].values
        ax.scatter(x_data, y_data, alpha=0.3, s=25,
                  color=dim_color, marker='s', edgecolors='red', linewidth=0.5)

    # Regression Gemini
    x_gem = df_gemini_clean[metric_x].dropna().values
    y_gem = df_gemini_clean[metric_y].dropna().values
    min_len = min(len(x_gem), len(y_gem))
    if min_len > 1:
        x_gem = x_gem[:min_len]
        y_gem = y_gem[:min_len]
        slope_gem, intercept_gem, r_gem, _, _ = stats.linregress(x_gem, y_gem)
        line_x = np.array([x_gem.min(), x_gem.max()])
        line_y = slope_gem * line_x + intercept_gem
        ax.plot(line_x, line_y, 'r--', linewidth=2.5, alpha=0.8, label=f'Gemini (r={r_gem:.2f})')

    ax.set_xlabel(label_x, fontsize=11, fontweight='bold')
    ax.set_ylabel(label_y, fontsize=11, fontweight='bold')
    ax.legend(fontsize=9, loc='best', framealpha=0.9)
    ax.grid(alpha=0.3)
    ax.tick_params(labelsize=9)

    if metric_x == "Conciseness":
        ax.set_xlim(0, 4)
    if metric_y == "Conciseness":
        ax.set_ylim(0, 4)
    if metric_x == "Num_addition":
        ax.set_xlim(0, 5)
    if metric_y == "Num_addition":
        ax.set_ylim(0, 5)
    if metric_x == "Num_direct_modifications":
        ax.set_xlim(0, 15)
    if metric_y == "Num_direct_modifications":
        ax.set_ylim(0, 15)

for idx in range(n_pairs, len(axes)):
    axes[idx].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.995])
plt.close()

Correlations Num vs other metrics

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# === FILES ===
files_gemini = {
    "GR": r"gr_gemini_ALL.csv",
    "SYNT": r"synt_gemini_ALL.csv",
    "READ": r"read_gemini_ALL.csv",
    "INFDENS": r"infdens_gemini_ALL.csv",
}

files_gpt = {
    "GR": r"gr_gpt_ALL.csv",
    "SYNT": r"synt_gpt_ALL.csv",
    "READ": r"read_gpt_ALL.csv",
    "INFDENS": r"infdens_gpt_ALL.csv",
}

metrics = ["acc_median_final", "comp_final_median", "key_concept_coverage", "Conciseness"]
controls = ["Num_addition", "Num_direct_modifications"]

display_names = {
    "Num_addition": "Num_addition",
    "Num_direct_modifications": "NumReplicas"
}

colors = {
    "GR": "#1f77b4",
    "SYNT": "#ff7f0e",
    "READ": "#2ca02c",
    "INFDENS": "#d62728"
}

output_base = r"grafici"

# --- Preprocessing ---
def preprocess_dataframe(df):
    if 'sentence_percentage' in df.columns:
        df = df.rename(columns={'sentence_percentage': 'percentage'})
    if 'setting' in df.columns:
        df['percentage_from_setting'] = df['setting'].map({'2s_3e': 50, '4s_3e': 100})
        if 'percentage' not in df.columns:
            df['percentage'] = df['percentage_from_setting']
        else:
            df['percentage'] = df['percentage'].fillna(df['percentage_from_setting'])
        df = df.drop(columns=['percentage_from_setting'])
    if 'percentage' in df.columns:
        df['percentage'] = pd.to_numeric(df['percentage'], errors='coerce')
    return df

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# --- Function to generate graphs with 50 and 100 side by side ---
def plot_correlations_50_100(files_dict, model_name):
    output_root = os.path.join(output_base, model_name.upper())
    ensure_dir(output_root)

    for ctrl in controls:
        for metric in metrics:
            fig, axes = plt.subplots(1, 2, figsize=(12,5), sharey=True)
            fig.suptitle(f"{metric} vs {display_names[ctrl]} ({model_name})", fontsize=14, fontweight='bold')

            for i, percentage_value in enumerate([50, 100]):
                ax = axes[i]
                ax.set_title(f"Percentage = {percentage_value}", fontsize=11, fontweight='bold')

                for name, path in files_dict.items():
                    df = pd.read_csv(path)
                    df = preprocess_dataframe(df)

                    if 'percentage' not in df.columns:
                        continue

                    df = df[df['percentage'] == percentage_value]
                    df = df.dropna(subset=[ctrl, metric])
                    if df.empty:
                        continue

                    X = df[ctrl]
                    Y = df[metric]

                    ax.scatter(X, Y, alpha=0.35, s=25, color=colors[name], label=name)
                    coeffs = np.polyfit(X, Y, 1)
                    trend = np.poly1d(coeffs)
                    ax.plot(np.sort(X), trend(np.sort(X)), color=colors[name], linewidth=2)

                ax.set_xlabel(display_names[ctrl])
                ax.grid(True, linestyle='--', alpha=0.3)
                if metric != "Conciseness":
                    ax.set_ylim(0,1)

                if i == 0:
                    ax.set_ylabel(metric)
                ax.legend(frameon=False, fontsize=8)

            plt.tight_layout(rect=[0,0,1,0.95])

            filename = f"{model_name}_{display_names[ctrl]}_{metric}_50_100.png"
            filepath = os.path.join(output_root, filename)
            plt.savefig(filepath, dpi=300, bbox_inches='tight')
            plt.close(fig)
            print(f"✅ Save: {filepath}")

# --- Esecuzione ---
plot_correlations_50_100(files_gemini, "Gemini")
plot_correlations_50_100(files_gpt, "GPT")
